In [1]:
# !pip install plotly matplotlib pandas requests huobi-client
# !pip list
# !git clone https://github.com/HuobiRDCenter/huobi_Python
# !cd huobi_Python && python setup.py install

In [2]:
import datetime
import requests
import pandas as pd
import random
from utils import *
from strategies import *
# from matplotlib import pyplot as plt

In [82]:
symbol_list = get_symbol_list()
sb = symbol_list[random.randint(0, len(symbol_list))]
print(sb)
# a=get_price("canusdt", "1min", 2000)
a=get_price(sb, "4hour", 2000)
a=get_rate(a, "close")
a["cr+1"] = (a["close"].shift(-1) - a["close"]) / a["close"]
a["cr+2"] = (a["close"].shift(-2) - a["close"]) / a["close"]
a=get_rate(a, "volume")
a=custom_data_analyze_func(a, rolling_days=3)
a=get_mean_line(a)
b=a.loc[a["cr+1"]>0.2]
display(a.loc[a["cr+1"]>0.2])
# draw_candle(a)

grtusdt


,high,low,open,close,volume,close_rate,cr+1,cr+2,volume_rate,close_cr,volume3_std,vol_mean3_cr,mean_5,mean_13,mean_21,mean_60,mean_120,mean_225
date,,,,,,,,,,,,,,,,,,
2020-12-18 08:00:00,0.146000,0.030000,0.030000,0.141668,2.468497e+06,NaN,0.253071,0.350940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-18 16:00:00,0.193000,0.170000,0.177946,0.191385,4.558149e+06,0.078104,0.334446,0.307030,-0.631172,0.078104,5.212531e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-19 04:00:00,0.264167,0.232000,0.250258,0.260920,6.542551e+06,0.043071,0.240062,0.278361,-0.657081,0.043071,1.646413e+07,-0.687543,0.227073,NaN,NaN,NaN,NaN,NaN
2020-12-19 12:00:00,0.335365,0.287829,0.324051,0.333550,1.203100e+07,0.030885,0.323556,0.406140,-0.492192,0.030885,8.757919e+06,-0.268092,0.284713,NaN,NaN,NaN,NaN,NaN
2020-12-20 08:00:00,0.538888,0.451000,0.477900,0.523281,1.891145e+07,0.094329,0.204208,0.299493,1.021393,0.094329,7.490133e+06,-0.468608,0.470364,0.329689,NaN,NaN,NaN,NaN
2020-12-26 16:00:00,0.353000,0.321774,0.352791,0.331137,2.304343e+06,-0.061009,0.361310,0.398956,0.018049,-0.061009,1.019777e+05,0.053284,0.344942,0.376090,0.405657,NaN,NaN,NaN
2021-02-12 08:00:00,2.120000,1.895551,2.039783,2.120000,1.555992e+07,0.039320,0.278725,0.249349,-0.206560,0.039320,2.358664e+07,-0.536107,1.892507,1.454054,1.276652,1.004338,0.798759,0.614609


In [90]:
dr_lis=[]
for dd in b.index:
    dr_lis.append((dd, dd-datetime.timedelta(hours=4*6)))

for dr in dr_lis:
    b=a.loc[(a.index >= dr[1]) & (a.index <= dr[0])]
    if len(b)>4:
        draw_candle(b)